# RAG Pipeline

You can use a retriever as part of a RAG (Retrieval-Augmented Generation) pipeline to provide context to a LLM.

In this lesson, you will use the vector retriever you created to pass additional context to an LLM allowing it to generate more accurate and relevant responses.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

from neo4j import GraphDatabase
from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings
from neo4j_graphrag.retrievers import VectorRetriever

# Connect to Neo4j database
driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI"), 
    auth=(
        os.getenv("NEO4J_USERNAME"), 
        os.getenv("NEO4J_PASSWORD")
    )
)

# Create embedder
embedder = OpenAIEmbeddings(model="text-embedding-ada-002")

# Create retriever
retriever = VectorRetriever(
    driver,
    index_name="moviePlots",
    embedder=embedder,
    return_properties=["title", "plot"],
)

# LLM

You will need an LLM to generate the response based on the user’s query and the context provided by the vector retriever.

The LLM is configured to use `gpt-4o`. You can change the configuration to another [OpenAI model](https://platform.openai.com/docs/models) by changing the model parameter.

You can also change the `temperature` to control the randomness of the generated response, by providing a value in the model_params. A value of 0 will make responses more deterministic, while a value closer to 1 will make responses more random.

The [neo4j-graphrag package](https://neo4j.com/docs/neo4j-graphrag-python/current/api.html?_gl=1*2x6016*_gcl_au*MjEzNTI4NjkxNy4xNzU3MjU4NDMzLjc4MDQ1OTczLjE3NTg0MTY3NjUuMTc1ODQxNjc2NA..*_ga*MTkzMzgxNTk1LjE3NTcyNTg0MzQ.*_ga_DL38Q8KGQC*czE3NjMyNjQ0MzMkbzQ3JGcxJHQxNzYzMjgxNDc2JGoyNiRsMCRoMA..*_ga_DZP8Z65KK4*czE3NjMyNjQ0MzMkbzQ3JGcxJHQxNzYzMjgxNDc2JGoyNiRsMCRoMA..#llm) supports multiple LLM models and the ability to create your own interface.

Create the LLM using the OpenAILLM class from the neo4j_graphrag package:

In [2]:
from neo4j_graphrag.llm import OpenAILLM

# Create the LLM
llm = OpenAILLM(model_name="gpt-4o")

## GraphRAG pipeline

The GraphRAG class allows you to create a RAG pipeline including a retriever and an LLM.

Create the GraphRAG pipeline using the retriever and the llm you created:

In [3]:
from neo4j_graphrag.generation import GraphRAG

# Create GraphRAG pipeline
rag = GraphRAG(retriever=retriever, llm=llm)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


## Search

You can use the search method to submit a query.

In [4]:
# Search
query_text = "Find me movies about toys coming alive"

response = rag.search(
    query_text=query_text, 
    retriever_config={"top_k": 5}
)

print(response.answer)

Here are a couple of movies about toys coming alive:

1. "Toy Story" - A cowboy doll is profoundly threatened and jealous when a new spaceman figure supplants him as top toy in a boy's room.
2. "The Indian in the Cupboard" - On his ninth birthday, a boy receives many presents, including a little Indian figure made of plastic that comes to life.


## Return context

You can also return the context that was used to generate the response. This can be useful in understanding how the LLM generated the response.

Add the `return_context=True` parameter to the `search` method:

In [5]:
# Search
query_text = "Find me movies about toys coming alive"

response = rag.search(
    query_text=query_text, 
    retriever_config={"top_k": 5},
    return_context=True
)

print(response.answer)
print("CONTEXT:", response.retriever_result.items)

Here are some movies about toys coming alive:

1. "Toy Story": A cowboy doll is profoundly threatened and jealous when a new spaceman figure supplants him as the top toy in a boy's room.
   
2. "The Indian in the Cupboard": On his ninth birthday, a boy receives a seemingly unimportant old cupboard and a little Indian figure made of plastic, which come to life.
CONTEXT: [RetrieverResultItem(content='{\'title\': \'Toy Story\', \'plot\': "A cowboy doll is profoundly threatened and jealous when a new spaceman figure supplants him as top toy in a boy\'s room."}', metadata={'score': 0.9163930416107178, 'nodeLabels': ['Movie'], 'id': '4:470cd2ae-a618-455c-b23a-7eaa0c9902c9:444'}), RetrieverResultItem(content="{'title': 'Jumanji', 'plot': 'When two kids find and play a magical board game, they release a man trapped for decades in it and a host of dangers that can only be stopped by finishing the game.'}", metadata={'score': 0.9066380262374878, 'nodeLabels': ['Movie'], 'id': '4:470cd2ae-a618-45

The `retriever_result` is returned along with the generated response.

Experiment with different queries and see how the LLM generates responses based on the context provided by the vector retriever.

Complete code

```python
import os
from dotenv import load_dotenv
load_dotenv()

from neo4j import GraphDatabase
from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings
from neo4j_graphrag.retrievers import VectorRetriever
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.generation import GraphRAG

# Connect to Neo4j database
driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI"), 
    auth=(
        os.getenv("NEO4J_USERNAME"), 
        os.getenv("NEO4J_PASSWORD")
    )
)

# Create embedder
embedder = OpenAIEmbeddings(model="text-embedding-ada-002")

# Create retriever
retriever = VectorRetriever(
    driver,
    index_name="moviePlots",
    embedder=embedder,
    return_properties=["title", "plot"],
)

# Create the LLM
llm = OpenAILLM(model_name="gpt-4o")


# Create GraphRAG pipeline
rag = GraphRAG(retriever=retriever, llm=llm)

# Search
query_text = "Find me movies about toys coming alive"

response = rag.search(
    query_text=query_text, 
    retriever_config={"top_k": 5}
)

print(response.answer)


# Close the database connection
driver.close()
```